# Tutorial 2

## Density Analysis

Understanding the particle density of a material is incredibly useful when studying things like defect segregation and atomic structure. Consider a system with a grain boundary, it may be interesting to know the structure changes at the boundary, e.g is there an increase or decrease in the amount of a certain species at the grain boundary and does this inform you about aby segregation behaviour? Another example involves a surface with a layer of water

The density module of polypy allows the particle density to be evaluated in one and two dimensions, this can then be converted into a charge density and (in one dimension) the electric field and electrostatic potential. 

In [1]:
from polypy.read import History
from polypy.density import Density
from polypy import analysis
from polypy import utils as ut
from polypy import plotting
import numpy as np
import matplotlib.pyplot as plt

The first step is to read the data. We want the data for both species so need to provide a list of the species.

```
["CA", "F"]
```

In [2]:
history = History("../example_data/HISTORY", ["CA", "F"])

The next step is to create the density object for both species.

In [3]:
ca_density = Density(history.trajectory, atom="CA")
f_density = Density(history.trajectory, atom="F")

## One Dimension

### Particle Density

The one_dimensional_density function will take a bin size e.g. 0.1 $AA$ and a direction e.g. the first lattice vector and calculate the total number of a species in 0.1 $AA$ bins in the first lattice vector.

The function will return the positions of the bins and the total number of species. These can then be plotted with the one_dimensional_plot function which takes a list of x values, a list of y values and a list of labels. 

In [4]:
cx, cy, c_volume = ca_density.one_dimensional_density(direction="z")
fx, fy, f_volume = f_density.one_dimensional_density(direction="z")

ax = plotting.one_dimensional_density_plot([fx, cx], [fy, cy], ["F", "Ca"])
plt.show()

NameError: name 'ca_density' is not defined

### Charge Density

The particle densities can be combined with the particle charges to generate the one dimensional charge density according to 

\begin{align}
\rho_q(z) = \sum_{i} q_i \rho_i(z)
\end{align}

where $\rho_{i}$ is the density of atom i and $q_{i}$ is its charge.  

The one_dimensional_charge_density function requires a list of particle densities, list of charges and the bin volume. 

In [ ]:
charge = analysis.OneDimensionalChargeDensity(fx, [fy, cy], [-1.0, 2.0], c_volume, history.trajectory.timesteps)

dx, charge_density = charge.calculate_charge_density()

ax = plotting.one_dimensional_charge_density_plot(dx, charge_density)
plt.show()

### Electric Field and Electrostatic Potential

The charge density can be converted into the electric field and the electrostatic potential.

\begin{align}
E(z) = \frac{1}{- \epsilon_{0}} \int_{z_{0}}^{z} \rho_{q}(z')dz'
\end{align}


\begin{align}
\Delta_{\psi}(z) = \int_{z_{0}}^{z} E(z')dz'
\end{align}

The poisson solver function will take the bin positions, and the charge density and return the electric field and the electrostatic potential. 

In [ ]:
dx, electric_field = charge.calculate_electric_field()
ax = plotting.electric_field_plot(dx, electric_field)
plt.show()

In [ ]:
dx, electrostatic_potential = charge.calculate_electrostatic_potential()

ax = plotting.electrostatic_potential_plot(dx, electrostatic_potential)
plt.show()


## Two Dimensions

### Particle Density

The particle density can be evaluated in two dimensions. The two_dimensional_density function will calculate the total number of species in cuboids. The coordinates in x and y of the box are returned and a grid of species counts are returned. 

In [ ]:
cx_2d, cy_2d, cz_2d, c_volume = ca_density.two_dimensional_density(direction="x")
fx_2d, fy_2d, fz_2d, f_volume = f_density.two_dimensional_density(direction="x")

fig, ax = plotting.two_dimensional_density_plot(cx_2d, cy_2d, cz_2d)
plt.show()

fig, ax = plotting.two_dimensional_density_plot(fx_2d, fy_2d, fz_2d)
plt.show()

### Charge Density

In the same fashion as the one dimensional case, the charge density can be evaluated in two dimensions using the two_dimensional_charge_density function. 

In [ ]:
charge_density = analysis.two_dimensional_charge_density([fz_2d, cz_2d], [-1.0, 2.0], f_volume)

fig, ax = plotting.two_dimensional_charge_density_plot(fx_2d, fy_2d, charge_density)
plt.show()

## One and Two Dimensions

Finally, the contour plots can give a good understanding of the average positions of the atoms (or the location of the lattice sites) however it does not give a good representation of how many species are actually there. The one_and_two_dimensional_overlay function will evaluate the particle dnesity in one and two dimensions and then overlay the two on to a single plot, allowing both the lattice sites, and total density to be viewed. 

In [ ]:
fig, ax = plotting.combined_density_plot(cx_2d, cy_2d, cz_2d)
plt.show()

fig, ax = plotting.combined_density_plot(fx_2d, fy_2d, fz_2d)
plt.show()